In [546]:
import pandas as pd
import numpy as np
import math

#fundamentals of the OLN, MRVL and AON
fundamentals = pd.read_csv('Data/funda OLN MRVL AON.csv')
ceq = fundamentals[['datadate','fyear','tic','ceq']].dropna()
ceq['datadate'] = ceq['datadate'].apply(lambda x:str(x)[:-2])

#Distribution of Book equity/Market equity for all the years
ME_BE_bp = pd.read_csv('Data/BE-ME_Breakpoints.csv',skiprows=2,skipfooter=1,names=list(np.arange(0,105,5))).dropna()
ME_BE_bp.reset_index(inplace=True)#.rename({'index':'date'},axis=1)#ME_bp[ME_bp['Date']=='200501']
ME_BE_bp.drop('level_1',inplace=True,axis=1)
ME_BE_bp = ME_BE_bp.rename({'level_0':'date'},axis=1)
ME_BE_bp['date'] = ME_BE_bp['date'].apply(lambda x:str(int(x))+'12')
ME_BE_bp[0] = ME_BE_bp[0].astype(int)

#Distribution of Market equity for all the years
ME_bp = pd.read_csv('Data/ME_Breakpoints.csv',skiprows=1,skipfooter=1,names=list(np.arange(0,105,5)))
ME_bp.reset_index(inplace=True)#.rename({'index':'date'},axis=1)#ME_bp[ME_bp['Date']=='200501']
ME_bp = ME_bp.rename({'index':'date'},axis=1)

#Returns of the 25 portfolios with combination of BE-ME and ME
port_return = pd.read_csv('Data/25_Portfolios_5x5.csv',skiprows=15,skip_blank_lines=True,skipfooter=1)
port_return['Unnamed: 0'] = port_return['Unnamed: 0'].apply(lambda x:"".join([s for s in str(x).split() if s.isdigit()])).dropna()
port_return = port_return.rename({'Unnamed: 0':'date'},axis=1).head(2284)
#port_return = port_return.astype(float)

#monthly prices of OLN, MRVL, AON
prices = pd.read_csv('Data/OLN MRVL AON.gz',compression='gzip')
prices['TICKER'] = prices['TICKER'].replace({'AOC':'AON'})#.dropna()
prices['MKTCAP'] = prices['PRC']*prices['SHROUT']/1000
prices['date'] = prices['date'].apply(lambda x:str(x)[:-2])

In [547]:
def f_ClosestVal(v_List, v_Number):
    """Takes an unsorted LIST of INTs and RETURNS INDEX of value closest to an INT"""
    for _index, i in enumerate(v_List):
        v_List[_index] = abs(v_Number - i)
    return v_List.index(min(v_List))

def get_returns(prices,date,ticker):
    return prices.groupby(['date','TICKER']).get_group((date,ticker))['RET'].astype(float).values[0]*100

In [548]:
def get_style(prices,ceq,date,ticker,style,percentile_table):
    mv = prices.groupby(['date','TICKER']).get_group((date,ticker))['MKTCAP'].values
    bv = ceq.groupby(['fyear','tic']).get_group((int(date[:-2]),ticker))['ceq'].values

    if style=='ME':
        me_bp_percentile = list(percentile_table[percentile_table['date'].astype(str)==date].values[0][1:])
        idx = f_ClosestVal(me_bp_percentile, mv)
        me_bp_percentile = list(percentile_table[percentile_table['date'].astype(str)==date].values[0][1:])

    if style=='BE/ME':
        me_bp_percentile = list(percentile_table[percentile_table['date'].astype(str)==date].values[0][1:])
        idx = f_ClosestVal(me_bp_percentile, list(bv/mv)[0])
        me_bp_percentile = list(percentile_table[percentile_table['date'].astype(str)==date].values[0][1:])
    
    return {'Date':date,
            'Ticker':ticker,
            'Market Value':mv[0],
            'Book Value':bv[0],
            'Book/Market':list(bv/mv)[0],
            'Nearest Percentile':me_bp_percentile[idx],
            'style':str(math.ceil(idx/4))+' '+style}

In [550]:
answer_df = pd.DataFrame.from_dict({0:get_style(prices,ceq,'200512','OLN','BE/ME',ME_BE_bp),
                                    1:get_style(prices,ceq,'200512','MRVL','BE/ME',ME_BE_bp),
                                    2:get_style(prices,ceq,'200512','AON','BE/ME',ME_BE_bp)},orient='index')
answer_df.insert(1,'Invested value $Million',[200,100,200])
answer_df.insert(8,'Returns',[get_returns(prices,'200601',tick) for tick in ['OLN','MRVL','AON']])
answer_df.head()

,Date,Invested value $Million,Ticker,Market Value,Book Value,Book/Market,Nearest Percentile,style,Returns
0,200412,200,OLN,1545.62784,356.00,0.230327,0.239,1 BE/ME,4.1667
1,200412,100,MRVL,9573.99146,2497.43,0.260856,0.239,1 BE/ME,21.9825
2,200412,200,AON,7550.61630,5103.00,0.675839,0.691,4 BE/ME,-4.3950


In [551]:
answer_df = pd.DataFrame.from_dict({0:get_style(prices,ceq,'200506','OLN','ME',ME_bp),
                                    1:get_style(prices,ceq,'200506','MRVL','ME',ME_bp),
                                    2:get_style(prices,ceq,'200506','AON','ME',ME_bp)},orient='index')
answer_df.insert(1,'Invested value $Million',[200,100,200])
answer_df.insert(8,'Returns',[get_returns(prices,'200601',tick) for tick in ['OLN','MRVL','AON']])
answer_df.head()

,Date,Invested value $Million,Ticker,Market Value,Book Value,Book/Market,Nearest Percentile,style,Returns
0,200506,200,OLN,1298.19552,426.600,0.328610,1321.18,2 ME,4.1667
1,200506,100,MRVL,10609.38755,3046.097,0.287113,11153.75,5 ME,21.9825
2,200506,200,AON,7958.11264,5303.000,0.666364,7548.01,4 ME,-4.3950
